<a href="https://colab.research.google.com/github/tomoyahiroe/transformers-playground/blob/main/tasks_that_need_to_insert_special_token_manually.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手動で特殊トークンを挿入しないといけないタスクの例

In [15]:
# transformersのインストール
!pip install transformers[ja,sentencepiece,torch]

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM
import pprint
import torch

## 穴埋めタスク

### T5(教科書3.4.4)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("retrieva-jp/t5-base-long")
model = AutoModelForSeq2SeqLM.from_pretrained("retrieva-jp/t5-base-long")

In [18]:
input_text = "江戸幕府を開いたのは、<extra_id_0>である。"
inputs = tokenizer(input_text, return_tensors="pt")

# 生成
outputs = model.generate(**inputs, max_length=3)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

徳川家康


### BERT(3.3.4)

In [22]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")
model = AutoModelForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-v3")

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

In [30]:
input_text = "江戸幕府を開いたのは、[MASK][MASK]である。"
inputs = tokenizer(input_text, return_tensors="pt")

# 生成
with torch.no_grad():
  outputs = model(**inputs)

# マスクされたトークンの予測を取得
masked_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
predicted_token_id = outputs.logits[0, masked_token_index].argmax(axis=-1)

# 予測されたトークンをデコードして表示
print(tokenizer.decode(predicted_token_id))

徳川 家康


#### 文法解説

In [33]:
inputs["input_ids"]

tensor([[    2, 13200, 14738,   500, 16371,   449,   464,   465,   384,     4,
             4,   457, 12485,   385,     3]])

In [34]:
inputs["input_ids"] == tokenizer.mask_token_id

tensor([[False, False, False, False, False, False, False, False, False,  True,
          True, False, False, False, False]])

In [36]:
# 出力 (tensor([0, 0]), tensor([ 9, 10])) は、バッチの 0 番目の要素の 9 番目と 10 番目の位置にマスクがあることを示しています。
torch.where(inputs["input_ids"] == tokenizer.mask_token_id)

(tensor([0, 0]), tensor([ 9, 10]))

In [37]:
torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

tensor([ 9, 10])

選択された outputs.logits[0, masked_token_index].argmax(axis=-1) は、モデルの予測結果から最も可能性の高い単語を特定する部分です。

outputs.logits は、モデルが各入力トークン位置に対して、語彙中の全単語がどれくらい適切であるかを示したスコア（ロジット）の集まりです。
[0, masked_token_index] は、このスコアの中から、最初の入力例（バッチサイズが 1 なので 0 番目）の、[MASK] トークンが見つかった位置（masked_token_index）に対応する部分を選び出しています。これにより、[MASK] の位置における、すべての単語に対するスコアのリストが得られます。
.argmax(axis=-1) は、そのスコアのリストの中で、最もスコアが高い（つまりモデルが一番予測している）単語の「インデックス」を見つけます。このインデックスは、語彙リストの中でのその単語の位置、つまり単語 ID になります。
したがって、このコードは「マスクされた位置について、モデルが予測した最も可能性の高い単語の ID を取得する」という処理を行っています。

In [40]:
outputs.logits[0, masked_token_index]

tensor([[-8.3264,  1.4076, -6.8395,  ..., -8.8488, -8.2784, -9.5503],
        [-8.4666,  2.9211, -5.8371,  ..., -9.0203, -8.2328, -9.6989]])

In [42]:
outputs.logits[0, masked_token_index].shape

torch.Size([2, 32768])

In [43]:
# 最も予測スコアの高い単語のインデックス
outputs.logits[0, masked_token_index].argmax(axis=-1)

tensor([15091, 17772])

In [44]:
tokenizer.decode(predicted_token_id)

'徳川 家康'